# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,falmouth,50.1544,-5.0711,13.50,100,53,1.03,GB,1691380545
1,1,badger,64.8000,-147.5333,25.95,50,100,0.00,US,1691380545
2,2,atlantis,-33.5667,18.4834,9.53,85,0,1.54,ZA,1691380546
3,3,adamstown,-25.0660,-130.1015,20.70,67,40,7.11,PN,1691380546
4,4,edinburgh of the seven seas,-37.0676,-12.3116,11.49,56,35,13.57,SH,1691380547


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display
# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng", 
    "Lat", 
    geo=True,
    tiles="OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 0.5,
    color = "City"
)
# Display the map plot
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_cities = city_data_df[(city_data_df['Max Temp'] > 15) & (city_data_df['Max Temp'] < 28)& \
                                    (city_data_df['Cloudiness'] >20) & (city_data_df['Humidity'] < 50)]

# Drop any rows with null values
ideal_weather_cities = ideal_weather_cities.dropna(how='any')

# Display sample data
ideal_weather_cities

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
20,20,caarapo,-22.6342,-54.8222,20.67,37,38,1.79,BR,1691380555
141,141,college,64.8569,-147.8028,25.99,49,100,0.00,US,1691380637
172,172,suntar,62.1444,117.6319,26.91,35,96,3.90,RU,1691380658
175,175,whitehorse,60.7161,-135.0538,27.43,29,75,6.17,CA,1691380662
179,179,hotan,37.0997,79.9269,27.30,33,81,6.14,CN,1691380664
210,210,jalai nur,49.4500,117.7000,22.42,42,95,6.19,CN,1691380687
288,288,kodinsk,58.6881,99.1844,20.71,43,100,2.92,RU,1691380733
299,299,tselinnoye,53.0833,85.6667,22.31,39,36,3.40,RU,1691380738
340,340,ola,59.5833,151.2833,16.75,47,100,2.81,RU,1691380760
367,367,priargunsk,50.3691,119.1012,24.39,43,63,3.70,RU,1691380775


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_cities.loc[:, ['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
20,caarapo,BR,-22.6342,-54.8222,37,
141,college,US,64.8569,-147.8028,49,
172,suntar,RU,62.1444,117.6319,35,
175,whitehorse,CA,60.7161,-135.0538,29,
179,hotan,CN,37.0997,79.9269,33,
210,jalai nur,CN,49.4500,117.7000,42,
288,kodinsk,RU,58.6881,99.1844,43,
299,tselinnoye,RU,53.0833,85.6667,39,
340,ola,RU,59.5833,151.2833,47,
367,priargunsk,RU,50.3691,119.1012,43,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey":geoapify_key,
    "format":"json"
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    response = requests.get(base_url,params=params)
    # Convert the API response to JSON format
    name_address = response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
caarapo - nearest hotel: No hotel found
college - nearest hotel: BEST WESTERN PLUS Pioneer Park Inn
suntar - nearest hotel: No hotel found
whitehorse - nearest hotel: Town & Mountain Hotel
hotan - nearest hotel: Hua Yi International Hotel (accepts foreigners)
jalai nur - nearest hotel: 满洲里福门客栈
kodinsk - nearest hotel: Ангара
tselinnoye - nearest hotel: Гостиница
ola - nearest hotel: No hotel found
priargunsk - nearest hotel: No hotel found
hinesville - nearest hotel: No hotel found
igarka - nearest hotel: Заполярье
coxim - nearest hotel: Avendita
kozhevnikovo - nearest hotel: Вега
estreito - nearest hotel: No hotel found
minot air force base - nearest hotel: No hotel found
temirlanovka - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
20,caarapo,BR,-22.6342,-54.8222,37,No hotel found
141,college,US,64.8569,-147.8028,49,BEST WESTERN PLUS Pioneer Park Inn
172,suntar,RU,62.1444,117.6319,35,No hotel found
175,whitehorse,CA,60.7161,-135.0538,29,Town & Mountain Hotel
179,hotan,CN,37.0997,79.9269,33,Hua Yi International Hotel (accepts foreigners)
210,jalai nur,CN,49.4500,117.7000,42,满洲里福门客栈
288,kodinsk,RU,58.6881,99.1844,43,Ангара
299,tselinnoye,RU,53.0833,85.6667,39,Гостиница
340,ola,RU,59.5833,151.2833,47,No hotel found
367,priargunsk,RU,50.3691,119.1012,43,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.loc[hotel_df["Hotel Name"]!="No hotel found"].hvplot.points(
    "Lng", 
    "Lat", 
    geo=True,
    tiles="OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 0.5,
    color = "City",
    hover_cols=["Hotel Name","Country"]
)
# Display the map plot
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)